In [5]:
# !pip install fastf1

In [15]:
import fastf1
import pandas as pd
import numpy as np

fastf1.Cache.enable_cache('cache')

# Loading data

Data - attributes from session <br>
[attr for attr in dir(session) if not attr.startswith('_')]

['api_path', <br>
 'car_data',<br>
 'date',<br>
 'drivers',<br>
 'event',<br>
 'f1_api_support',<br>
 'get_circuit_info',<br>
 'get_driver',<br>
 'laps',<br>
 'load',<br>
 'name',<br>
 'pos_data',<br>
 'race_control_messages',<br>
 'results',<br>
 'session_info',<br>
 'session_start_time',<br>
 'session_status',<br>
 't0_date',<br>
 'total_laps',<br>
 'track_status',<br>
 'weather_data']

In [12]:
all_laps = []
all_drivers = []
all_events = []
all_weather = []
all_track_status = []

for season in range(2020, 2025):
    calendar = fastf1.get_event_schedule(season)

    for _, row in calendar.iterrows():
        try:
            session = fastf1.get_session(season, row['Country'], 'R')
    
            session.load(
                telemetry=False,
                messages=False
            )
    
            # --- EVENT / TRACK INFO ---
            event = session.event.copy()
            event['Season'] = season
            all_events.append(event)
    
            # --- DRIVERS ---
            for drv in session.drivers:
                d = session.get_driver(drv)
                d['Season'] = season
                d['Round'] = row['RoundNumber']
                all_drivers.append(d)
            
            # --- LAPS ---
            laps = session.laps.copy()
            laps['Season'] = season
            laps['Round'] = row['RoundNumber']
            laps['EventName'] = row['EventName']
            all_laps.append(laps)
            
            # --- WEATHER ---
            if session.weather_data is not None:
                weather = session.weather_data.copy()
                weather['Season'] = season
                weather['Round'] = row['RoundNumber']
                all_weather.append(weather)
    
            # --- TRACK STATUS ---
            if session.track_status is not None:
                ts = session.track_status.copy()
                ts['Season'] = season
                ts['Round'] = row['RoundNumber']
                all_track_status.append(ts)
                
        except Exception as e:
            print(f"Skipped {season} Round {row['RoundNumber']}->{row['EventName']}: {e}")

logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	Using cached data for season_schedule
events      WARNING 	Correcting user input 'Austria' to 'Austrian Grand Prix'
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver  8: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for weather_data
core

ValueError: Failed to load any schedule data.

In [13]:
df_laps = pd.concat(all_laps, ignore_index=True)
df_weather = pd.concat(all_weather, ignore_index=True)
df_track_status = pd.concat(all_track_status, ignore_index=True)
df_drivers = pd.DataFrame(all_drivers)

In [16]:
df_laps.head()

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Season,Round,EventName
0,0 days 00:34:06.302000,GAS,10,0 days 00:01:19.106000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:33.352000,...,NaT,1,12.0,None,,False,False,2020,1,Austrian Grand Prix
1,0 days 00:35:18.714000,GAS,10,0 days 00:01:12.412000,2.0,1.0,NaT,NaT,0 days 00:00:17.960000,0 days 00:00:32.228000,...,NaT,1,12.0,None,,False,True,2020,1,Austrian Grand Prix
2,0 days 00:36:30.025000,GAS,10,0 days 00:01:11.311000,3.0,1.0,NaT,NaT,0 days 00:00:17.513000,0 days 00:00:31.717000,...,NaT,1,12.0,None,,False,True,2020,1,Austrian Grand Prix
3,0 days 00:37:40.750000,GAS,10,0 days 00:01:10.725000,4.0,1.0,NaT,NaT,0 days 00:00:17.410000,0 days 00:00:31.510000,...,NaT,1,12.0,None,,False,True,2020,1,Austrian Grand Prix
4,0 days 00:38:52.261000,GAS,10,0 days 00:01:11.511000,5.0,1.0,NaT,NaT,0 days 00:00:17.464000,0 days 00:00:31.827000,...,NaT,1,12.0,None,,False,True,2020,1,Austrian Grand Prix
